# waf

help(wsh.xarray.VectorWind)

In [7]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
print('BEGIN')

import socket
import re
import os
import sys
import inspect
import pickle
import glob
#import sort
import datetime
import socket
import re
import os
import sys
import netCDF4
import math

print('sys.version=',sys.version)

hostname=socket.gethostname()

print('hostname=',hostname)

if(re.match('raijin',hostname)):
  print('this is rajin')
  rundir='/short/v14/mac599/cafepp/rundir'
  topdir=''
elif(re.match('oa-3.-cdc',hostname)):
  print('this is vm32')
  rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
  topdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6'
  topdir='/OSM/CBR/OA_DCFP/work/col414/CAFEPP/CMIP6'
  topdir_vm='/OSM/CBR/OA_DCFP/data/CAFEPP'
  topdir_vm='/OSM/CBR/OA_DCFP/work/col414/CAFEPP'
  
os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import numpy as np
import numpy.ma as ma
from matplotlib.mlab import griddata
from scipy import signal
from matplotlib import pyplot as plt
from PyAstronomy import pyaC
import xarray as xr
import windspharm as wsh
import pandas as pd

from decadal_diag import \
  fractional_year_from_num2date, \
  generate_daily_month_indices, \
  shade_2d_latlon, \
  get_timestamp_number, \
  shade_2d_curvilinear, \
  shade_2d_simple, \
  nino_indices, \
  plot_xy, \
  fractional_year_from_num2date, \
  convert_bytes, \
  file_size, \
  compress_nc, \
  prepare_vertintp, \
  get_exitcode_stdout_stderr, \
  box_indices, \
  plot_box_indices, \
  basic_stats, \
  lagcorr, \
  turning_points, \
  plot_bar
  
from other import year_fraction

from n_data_funcs import n_data_funcs

from doppyo import skill
from doppyo import utils
from doppyo import diagnostic

os.chdir('/OSM/CBR/OA_DCFP/work/squ027/doppyo/doppyo')

print('Current Working Directory=',os.getcwd())

cafe_experiment_titles = {1:'v0_forecast', 4:'v1_forecast', 3:'v2_forecast', 9:'v1', \
  5:'v2', 6:'v3', 7:'aug17p', 2:'v1_assim', 10:'etkf', 11:'nov17n', 12:'jul18b', 13:'v0'}

fluxes_colors={'hfls':'red', 'hfss':'blue', 'rlds':'green', 'rlus':'orange', \
               'rlut':'pink', 'rsdt':'brown', 'rsus':'cyan', 'rsds':'purple', 'rsut':'grey', \
              'net down toa':'tan', 'net down surf':'yellow'}

%matplotlib inline

CRED = '\033[91m'
CEND = '\033[0m'

#print(cafe_experiment_titles[1])

print('END')

BEGIN
sys.version= 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 17:14:51) 
[GCC 7.2.0]
hostname= oa-35-cdc
this is vm32
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
Current Working Directory= /OSM/CBR/OA_DCFP/work/squ027/doppyo/doppyo
END


In [9]:
u = xr.DataArray(np.random.normal(size=(6,4)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
v = xr.DataArray(np.random.normal(size=(6,4)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
diagnostic.velocity_potential(u, v)


<xarray.DataArray 'phi' (lat: 6, lon: 4)>
array([[  785062.9 ,   785062.9 ,   785062.9 ,   785062.9 ],
       [  775151.5 ,   831175.1 ,  1032900.25,   976876.6 ],
       [ -501071.3 ,   317162.38,   614048.5 ,  -204185.12],
       [ -887251.56,   735306.6 ,    11376.75, -1611181.4 ],
       [ -238267.38,  -319824.62,  -340628.88,  -259071.62],
       [ -613033.  ,  -613033.  ,  -613033.  ,  -613033.  ]], dtype=float32)
Coordinates:
  * lat      (lat) int64 75 45 15 -15 -45 -75
  * lon      (lon) int64 45 135 225 315
Attributes:
    units:          m**2 s**-1
    standard_name:  atmosphere_horizontal_velocity_potential
    long_name:      velocity potential

In [16]:
u = xr.DataArray(np.random.normal(size=(6,4,2,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [100,500]), 
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
v = xr.DataArray(np.random.normal(size=(6,4,2,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [100,500]), \
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
u_clim = u.groupby('time.month').mean(dim='time')
v_clim = v.groupby('time.month').mean(dim='time')

print('u_clim.shape=',u_clim.shape)
print('u_clim=',u_clim)

u_anom = utils.anomalize(u, u_clim)
v_anom = utils.anomalize(v, v_clim)
psi_anom = diagnostic.stream_function(u_anom, v_anom)
out = diagnostic.wave_activity_flux(psi_anom, u, v)

print('out=',out)

print('END')

u_clim.shape= (6, 4, 2, 12)
u_clim= <xarray.DataArray (lat: 6, lon: 4, level: 2, month: 12)>
array([[[[ 0.555334, ...,  0.15984 ],
         [ 0.468287, ..., -0.994917]],

        ...,

        [[-0.452323, ..., -0.615794],
         [-0.221831, ...,  0.391634]]],


       ...,


       [[[-0.879602, ..., -1.317467],
         [ 0.774172, ...,  0.238517]],

        ...,

        [[ 1.133758, ..., -0.750029],
         [ 0.580929, ...,  0.023616]]]])
Coordinates:
  * lat      (lat) int64 -75 -45 -15 15 45 75
  * lon      (lon) int64 45 135 225 315
  * level    (level) int64 100 500
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
w= <windspharm.xarray.VectorWind object at 0x7ff547e07c88>
out= <xarray.Dataset>
Dimensions:  (lat: 6, level: 2, lon: 4, time: 24)
Coordinates:
  * level    (level) int64 100 500
  * lat      (lat) int64 -75 -45 -15 15 45 75
  * lon      (lon) int64 45 135 225 315
  * time     (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2001-12-31
Data variables:
    u_waf

In [ ]:
u = xr.DataArray(np.random.normal(size=(6,4)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
v = xr.DataArray(np.random.normal(size=(6,4)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90))])
out = diagnostic.Rossby_wave_source(u, v)

print('out=',out)

print('END')

In [6]:
u = xr.DataArray(np.random.normal(size=(6,4,2,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [100,500]), \
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
v = xr.DataArray(np.random.normal(size=(6,4,2,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [100,500]), \
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
u_clim = u.groupby('time.month').mean(dim='time')
v_clim = v.groupby('time.month').mean(dim='time')
u_anom = utils.anomalize(u, u_clim)
v_anom = utils.anomalize(v, v_clim)
psi_anom = diagnostic.stream_function(u_anom, v_anom)
data=diagnostic.wave_activity_flux(psi_anom, u, v)

print('data=',data)

print('END')

w= <windspharm.xarray.VectorWind object at 0x7ff584f6f908>
data= <xarray.Dataset>
Dimensions:  (lat: 6, level: 2, lon: 4, time: 24)
Coordinates:
  * level    (level) int64 100 500
  * lat      (lat) int64 -75 -45 -15 15 45 75
  * lon      (lon) int64 45 135 225 315
  * time     (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2001-12-31
Data variables:
    u_waf    (level, lat, lon, time) float64 0.008912 0.002964 ... 0.165 -0.0359
    v_waf    (level, lat, lon, time) float64 -0.0025 0.01668 ... 0.003272
END


In [3]:
u = xr.DataArray(np.random.normal(size=(6,4,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
v = xr.DataArray(np.random.normal(size=(6,4,24)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('time', pd.date_range('2000-01-01',periods=24,freq='M'))])
u_clim = u.groupby('time.month').mean(dim='time')
u_clim = utils.anomalize(0*u, -u_clim)
print(diagnostic.Rossby_wave_number(u, v, u_clim))

print('END')

<xarray.DataArray 'ks2' (lat: 6, lon: 4, time: 24)>
array([[[ 6.238878e+00, -1.079475e+00, ..., -1.304777e+00, -2.375866e-01],
        [ 4.370043e-01,  4.256758e+00, ...,  3.225474e-01, -2.578716e+01],
        [ 1.077790e+00,  2.937898e+00, ..., -5.718076e-01,  1.147554e-01],
        [-4.034708e-01,  7.197462e+00, ...,  4.871033e+01,  1.699499e+00]],

       [[-8.343896e+02, -8.254493e+02, ..., -5.613415e+02, -2.169745e+03],
        [ 1.084508e+03, -6.348874e+02, ...,  1.906387e+02, -7.279070e+02],
        [ 9.110567e+02, -7.194693e+02, ...,  3.255692e+02, -2.988381e+02],
        [-4.561224e+02,  6.642433e+02, ..., -5.895170e+02,  2.701369e+03]],

       ...,

       [[-3.676990e+03,  3.309632e+02, ...,  4.501356e+03,  7.693665e+02],
        [-3.730192e+02,  1.023825e+03, ..., -1.252109e+04, -2.930880e+02],
        [ 1.262031e+03, -1.949834e+03, ..., -1.288571e+04, -2.729175e+02],
        [-4.832188e+02,  3.634579e+03, ..., -1.077926e+03,  4.754549e+02]],

       [[-7.460437e+00, -5.88

In [ ]:
u = xr.DataArray(np.random.normal(size=(6,4,2)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [200, 500])])
v = xr.DataArray(np.random.normal(size=(6,4,2)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [200, 500])])
temp = xr.DataArray(np.random.normal(size=(6,4,2)), \
  coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
  ('level', [200, 500])])
gh = xr.DataArray(np.random.normal(size=(6,4,2)), \
coords=[('lat', np.arange(-75,76,30)), ('lon', np.arange(45,316,90)), \
('level', [200, 500])])
nsq = diagnostic.Brunt_Vaisala(temp)
print(diagnostic.Eady_growth_rate(u, v, gh, nsq))

print('END')

In [ ]:
gh = xr.DataArray(np.random.normal(size=(3,4,4)), \
coords=[('level', [400, 500, 600]), ('lat', np.arange(-90,90,45)), \
('lon', np.arange(0,360,90))])
print(diagnostic.thermal_wind(gh, plevel_lower=400, plevel_upper=600))

print('END')

In [ ]:
v = xr.DataArray(np.random.normal(size=(2,4,4)), \
  coords=[('level', [400, 600]), ('lat', np.arange(-90,90,45)), \
  ('lon', np.arange(0,360,90))])

print(diagnostic.mean_merid_mass_streamfunction(v))

print('END')